In [ ]:
!pip install transformers torch langdetect googletrans==4.0.0-rc1 indic-transliteration

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 981.5/981.5 kB 9.8 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
  Preparing metadata (setup.py) ... done
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 55.1/55.1 kB 3.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 133.4/133.4 kB 9.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 42.6/42.6 kB 1.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 58.8/58.8 kB 3.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 65.0/65.0 kB 4.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 363.4/363.4 MB 3.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 13.8/13.8 MB 53.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 24.6/24.6 MB 48.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 883.7/883.7 kB 30.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 664.8/664.8 MB 2.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 21

In [ ]:
from transformers import AutoTokenizer, AutoModelForSequenceClassification
import torch
from langdetect import detect, DetectorFactory
from googletrans import Translator
from indic_transliteration import sanscript
from indic_transliteration.sanscript import transliterate


In [ ]:
# To make language detection consistent
DetectorFactory.seed = 0


In [ ]:
MODEL_NAME = "unitary/toxic-bert"
tokenizer = AutoTokenizer.from_pretrained(MODEL_NAME)
model = AutoModelForSequenceClassification.from_pretrained(MODEL_NAME)

TOXIC_LABELS = ["toxicity", "severe_toxicity", "obscene", "identity_attack", "insult", "threat"]
translator = Translator()


/usr/local/lib/python3.11/dist-packages/huggingface_hub/utils/_auth.py:94: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


tokenizer_config.json:   0%|          | 0.00/174 [00:00<?, ?B/s]

config.json:   0%|          | 0.00/811 [00:00<?, ?B/s]

vocab.txt:   0%|          | 0.00/232k [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/112 [00:00<?, ?B/s]

model.safetensors:   0%|          | 0.00/438M [00:00<?, ?B/s]

In [ ]:
def is_probably_tamil_roman(text):
    tamil_roman_keywords = ['enna', 'illa', 'podu', 'poda', 'dei', 'machan', 'madiri', 'romba', 'unga', 'evlo', 'aama']
    text_lower = text.lower()
    return any(word in text_lower for word in tamil_roman_keywords)


In [ ]:
def translate_to_english(text):
    try:
        detected_lang = detect(text)
    except:
        detected_lang = "unknown"

    # Step 1: Check for Tunglish
    if is_probably_tamil_roman(text):
        print("🔤 Detected as Tunglish")
        tamil_script = transliterate(text, sanscript.ITRANS, sanscript.TAMIL)
        print(f"📝 Transliterated to Tamil: {tamil_script}")
        translated = translator.translate(tamil_script, src='ta', dest='en')
        return translated.text, 'tunglish'

    # Step 2: Force English if short and likely valid
    if detected_lang != "en" and len(text.split()) <= 4 and all(char.isalpha() or char.isspace() for char in text):
        print("🔤 Detected as English (forced due to short clean sentence)")
        return text, 'en'

    # Step 3: Regular route
    if detected_lang == "en":
        return text, 'en'
    elif detected_lang != "unknown":
        translated = translator.translate(text, src=detected_lang, dest='en')
        return translated.text, detected_lang
    else:
        return text, 'unknown'



In [ ]:
def predict_toxicity(text, threshold=0.5):
    eng_text, lang = translate_to_english(text)

    inputs = tokenizer(eng_text, return_tensors="pt", truncation=True, padding=True)
    with torch.no_grad():
        outputs = model(**inputs)
        scores = torch.sigmoid(outputs.logits)[0].tolist()

    result = {
        "original_language": lang,
        "translated_text": eng_text
    }

    is_toxic = False
    for label, score in zip(TOXIC_LABELS, scores):
        result[label] = round(score, 4)
        if score >= threshold:
            is_toxic = True

    result["final_decision"] = "Blocked" if is_toxic else "Allowed"
    return result


In [ ]:
user_comment = input("🗣️ Enter your comment (English, Tamil or Tunglish): ")
output = predict_toxicity(user_comment)

print(f"\n🌐 Original Language: {output['original_language']}")
print(f"🔁 Translated to English: {output['translated_text']}")
print("📊 Toxicity Scores:")
for label in TOXIC_LABELS:
    print(f"  {label}: {output[label]}")
print(f"\n🚦 Final Result: {output['final_decision']}")


🗣️ Enter your comment (English, Tamil or Tunglish): i hate it
🔤 Detected as English (forced due to short clean sentence)

🌐 Original Language: en
🔁 Translated to English: i hate it
📊 Toxicity Scores:
  toxicity: 0.4038
  severe_toxicity: 0.0007
  obscene: 0.0046
  identity_attack: 0.0029
  insult: 0.0029
  threat: 0.0024

🚦 Final Result: Allowed
